# Milestone 1 : Regression - Predicting seismic collapse capacity

#### Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import metrics

## 1.Data

### 1.1 Import

In [2]:
df = {}
for t in ['train', 'test', 'val'] :
    df[t] = pd.read_csv(f"Data/{t}_set.csv")
df['train'].head(5)

,0.01,0.02,0.022,0.025,0.029,0.03,0.032,0.035,0.036,0.04,...,8.5,9.0,9.5,10.0,sa_avg,da5_75,da5_95,fiv3,sa_ratio,sat1_col
0,0.011075,0.011107,0.011132,0.011176,0.011274,0.011288,0.011341,0.011377,0.011422,0.011567,...,0.000247,0.000223,0.000206,0.000190,0.010809,14.484,21.416,2.881797,0.832237,0.78
1,0.155380,0.153050,0.154590,0.156281,0.161038,0.160171,0.159765,0.166164,0.173643,0.174708,...,0.002316,0.002225,0.002106,0.001998,0.031044,14.430,18.270,7.642059,0.961638,1.96
2,0.060774,0.060783,0.060790,0.060795,0.060799,0.060800,0.060804,0.060794,0.060793,0.060798,...,0.013225,0.012226,0.011904,0.011696,0.098425,16.700,35.105,42.218868,1.737888,1.39
3,0.016016,0.016088,0.016106,0.016135,0.016198,0.016218,0.016263,0.016401,0.016445,0.016595,...,0.000192,0.000163,0.000139,0.000124,0.010169,7.320,17.370,2.599605,1.000551,1.97
4,0.030632,0.030699,0.030724,0.030738,0.030785,0.030809,0.030811,0.030818,0.030856,0.031014,...,0.004094,0.003350,0.002638,0.002224,0.037375,28.005,41.635,11.434507,1.371310,1.43


#### Columns options :

In [3]:
X_col = {}
y_col = ['sat1_col']
#Option 1 : All
X_col["All"] = df['test'].columns.to_numpy()
#Option 2 : Reduce to last columns (no sa(T), only the other columns. sa_ratio and sa_avg is there to convey the sa information)
X_col["Reduced"] = ['1.3','sa_avg','da5_75', 'da5_95', 'fiv3', 'sa_ratio']
#Option 3 : Best 15 cols (section 7.)
X_col['Best_15'] = ['sa_avg', 'fiv3', 'da5_95', '0.06', '0.1', 'max_period', 'da5_75', '1.5', '0.07'
, '0.34', '0.32', '0.03', '0.4', '0.13', '9.0']

### Data Expansion : Max Period

In [4]:
for t in ['train', 'test', 'val'] :
    df[t]['max_period'] = df[t][df['test'].columns.difference(X_col["Reduced"]+["max_period"])].idxmax(axis="columns").astype("float")

In [5]:
df['val']['sat1_col']

0       1.65
1       1.29
2       1.40
3       1.90
4       2.34
        ... 
1495    1.00
1496    1.00
1497    1.26
1498    1.30
1499    1.54
Name: sat1_col, Length: 1500, dtype: float64

In [6]:
#Option 3 and 3.1 : Add max_period
X_col["Red_max_period"] = ['sa_avg','da5_75', 'da5_95', 'fiv3', 'sa_ratio', 'max_period']
X_col["All_max_period"] = df['test'].columns.to_numpy()

### Choice of parameters

In [7]:
#Choix :
columns = "All_max_period"

In [8]:
X_train = df['train'][X_col[columns]].to_numpy()
y_train = df['train'][y_col].to_numpy()

X_val = df['val'][X_col[columns]].to_numpy()
y_val = df['val'][y_col].to_numpy()

X_test = df['test'][X_col[columns]].to_numpy()

cols_map = np.array(X_col[columns])

print(X_train.shape)
print(X_test.shape)
print(X_val.shape)
print(cols_map)

(12646, 111)
(3000, 111)
(1500, 111)
['0.01' '0.02' '0.022' '0.025' '0.029' '0.03' '0.032' '0.035' '0.036'
 '0.04' '0.042' '0.044' '0.045' '0.046' '0.048' '0.05' '0.055' '0.06'
 '0.065' '0.067' '0.07' '0.075' '0.08' '0.085' '0.09' '0.095' '0.1' '0.11'
 '0.12' '0.13' '0.133' '0.14' '0.15' '0.16' '0.17' '0.18' '0.19' '0.2'
 '0.22' '0.24' '0.25' '0.26' '0.28' '0.29' '0.3' '0.32' '0.34' '0.35'
 '0.36' '0.38' '0.4' '0.42' '0.44' '0.45' '0.46' '0.48' '0.5' '0.55' '0.6'
 '0.65' '0.667' '0.7' '0.75' '0.8' '0.85' '0.9' '0.95' '1.0' '1.1' '1.2'
 '1.3' '1.4' '1.5' '1.6' '1.7' '1.8' '1.9' '2.0' '2.2' '2.4' '2.5' '2.6'
 '2.8' '3.0' '3.2' '3.4' '3.5' '3.6' '3.8' '4.0' '4.2' '4.4' '4.6' '4.8'
 '5.0' '5.5' '6.0' '6.5' '7.0' '7.5' '8.0' '8.5' '9.0' '9.5' '10.0'
 'sa_avg' 'da5_75' 'da5_95' 'fiv3' 'sa_ratio' 'max_period']


### 1.2 Normalisation

In [9]:
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)

def Normalise(X):
    return (X-mean)/std

X_train = Normalise(X_train)
X_val = Normalise(X_val)
X_test = Normalise(X_test)



### 1.3 Import en Pytorch

https://stackoverflow.com/questions/44429199/how-to-load-a-list-of-numpy-arrays-to-pytorch-dataset-loader

In [10]:
def convert_to_dataloader(x, y=None, batch_size = 10):
    tensor_x = torch.Tensor(x)
    try:
        if y == None:
            dataset = torch.utils.data.TensorDataset(tensor_x)
    except:
        tensor_y = torch.Tensor(y)
        dataset = torch.utils.data.TensorDataset(tensor_x,tensor_y)
    return torch.utils.data.DataLoader(dataset, batch_size = batch_size)

dataload_train = convert_to_dataloader(X_train, y_train)
dataload_val = convert_to_dataloader(X_val, y_val)
dataload_test = convert_to_dataloader(X_test)

## 2. Models

### 2.1 OneLayerNet

In [11]:
class OneLayerNet(nn.Module):
    """1-Layer linear"""
    
    def __init__(self, cols):
        super().__init__()
        self.fc1 = nn.Linear(cols, 1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x.flatten(start_dim=1)
        x = self.fc1(x)
        return x
  
    def predict(self, x: torch.Tensor) -> torch.Tensor:
        y = self.forward(x)
        return y

one_layer_net = OneLayerNet(X_train.shape[1])

### TwoLayerNet

In [ ]:
class TwoLayerNet(nn.Module):
    """2-Layer linear+RELU"""
    
    def __init__(self, cols):
        super().__init__()
        self.fc1 = nn.Linear(cols, cols//3)
        self.fc2 = nn.Linear(cols//3, 1)
     

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x.flatten(start_dim=1)
        x = self.fc1(x)
        x = self.fc2(F.LeakyReLU(x))
        return x
  
    def predict(self, x: torch.Tensor) -> torch.Tensor:
        y = self.forward(x)
        return y

two_layer_net = TwoLayerNet(X_train.shape[1])

### 2.2 ThreeLayerNet

In [12]:
class ThreeLayerNet(nn.Module):
    """3-Layer linear+RELU"""
    
    def __init__(self, cols):
        super().__init__()
        self.fc1 = nn.Linear(cols, cols//2)
        self.fc2 = nn.Linear(cols//2, cols//4)
        self.fc3 = nn.Linear(cols//4, 1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x.flatten(start_dim=1)
        x = self.fc1(x)
        x = self.fc2(F.relu(x))
        x = self.fc3(F.relu(x))
        return x
  
    def predict(self, x: torch.Tensor) -> torch.Tensor:
        y = self.forward(x)
        return y

three_layer_net = ThreeLayerNet(X_train.shape[1])

### 2.3 FourLayerNet

In [13]:
class FourLayerNet(nn.Module):
    """4-Layer linear+RELU + Non-linear"""
    
    def __init__(self, cols):
        super().__init__()
        self.fc1 = nn.Linear(cols, cols//2)
        self.fc2 = nn.Linear(cols//2, cols//4)
        self.fc3 = nn.Linear(2*(cols//4), cols//4)
        self.fc4 = nn.Linear(cols//4, 1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x.flatten(start_dim=1)
        x = self.fc1(x)
        x = self.fc2(F.gelu(x))
        x2 = x**2
        x = torch.stack([x, x2], dim=2).flatten(start_dim=1) # Adding x^2 terms
        x = self.fc3(F.relu(x))
        x = self.fc4(F.relu(x))
        return x

    def predict(self, x: torch.Tensor) -> torch.Tensor:
        y = self.forward(x)
        return y

four_layer_net = FourLayerNet(X_train.shape[1])

### 2.4 NonLinearNet

Very Bad Result : Loss goes in dent de scie

In [14]:
class NonLinearNet(nn.Module):
    """Non-linear (first)"""
    
    def __init__(self, cols):
        super().__init__()
        self.fc1 = nn.Linear(cols, cols//4)
        self.fc2 = nn.Linear(4*(cols//4), cols//2)
        self.fc3 = nn.Linear(cols//2, cols//8)
        self.fc4 = nn.Linear(cols//8, 1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x.flatten(start_dim=1)
        x = F.relu(self.fc1(x))
        x = torch.stack([x, x**2, torch.log(x+1e-3), torch.exp(x)], dim=2).flatten(start_dim=1) # Adding x^2, log, exp terms
        x = self.fc2(F.gelu(x))
        x = self.fc3(F.relu(x))
        x = self.fc4(F.relu(x))
        return x

    def predict(self, x: torch.Tensor) -> torch.Tensor:
        y = self.forward(x)
        return y

non_linear_net = NonLinearNet(X_train.shape[1])

In [15]:
class NonLinearNet2(nn.Module):
    """Non-linear (second)"""
    
    def __init__(self, cols):
        super().__init__()
        self.fc1 = nn.Linear(3*cols, cols)
        self.fc2 = nn.Linear(cols, cols//2)
        self.fc3 = nn.Linear(cols//2, cols//4)
        self.fc4 = nn.Linear(cols//4, 1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x.flatten(start_dim=1)
        x2 = torch.hstack((x[:,1:]*x[:,:-1], (x[:,1]*x[:,-1]).view(-1,1))) #xy terms
        x = torch.hstack([x, x**2, x2]) #Adding x^2, xy terms
        x = self.fc1(x)
        x = self.fc2(F.gelu(x))
        x = self.fc3(F.relu(x))
        x = self.fc4(F.relu(x))
        return x

    def predict(self, x: torch.Tensor) -> torch.Tensor:
        y = self.forward(x)
        return y

non_linear_net2 = NonLinearNet2(X_train.shape[1])

## 3. Training

### 3.1 Loss function

In [16]:
loss_fn = nn.MSELoss(reduction="mean")

### 3.2 Training

In [17]:
def train(model: torch.nn.Module, train_loader: torch.utils.data.DataLoader, loss_fn: torch.nn.Module, optimizer: torch.optim.Optimizer, epochs: int):
    
    # Initialize metrics for loss and accuracy
    #loss_metric = metrics.LossMetric()
    
    # Sets the module in training mode (doesn't have any effect here, but good habit to take)
    model.train()
    
    for epoch in range(1, epochs + 1):
        losses = []
        # Progress bar set-up
        #pbar = tqdm(total=len(train_loader), leave=True)
        #pbar.set_description(f"Epoch {epoch}")
        
        # Iterate through data
        for data, target in train_loader:
            
            ### START CODE HERE ###
            
            # Zero-out the gradients
            optimizer.zero_grad()
            
            # Forward pass
            out = model(data)
            
            # Compute loss
            loss = loss_fn(out, target)
            
            # Backward pass
            loss.backward()
            
            # Optimizer step
            optimizer.step()
            
            ### END CODE HERE ###
            
            # Update metrics & progress bar
            #loss_metric.update(loss.item(), data.shape[0])
            #pbar.update()
            losses.append(loss.item())
            
        # End of epoch, show loss and acc
        #pbar.set_postfix_str(f"Train loss: {loss_metric.compute():.3f} | Train acc: {acc_metric.compute() * 100:.2f}%")
        #print(f"Train loss: {loss_metric.compute():.3f}")
        #loss_metric.reset()

        print(epoch, np.mean(losses))

In [35]:
model = four_layer_net
#optimizer = optim.SGD(model.parameters(), lr=0.001)
#optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4)
optimizer = torch.optim.AdamW(model.parameters(), )
train(model, dataload_train, loss_fn, optimizer, epochs=500)

1 0.27905559891341586
2 0.1802843687798195
3 0.17702456532568916
4 0.1727237864864968
5 0.17207399764919115
6 0.1706434350755257
7 0.1684641240017393
8 0.1674298066123142
9 0.16815398664695944
10 0.16503701230488982
11 0.1645396400913007
12 0.1651534841819243
13 0.1629695195910841
14 0.16204030238563127
15 0.16136358982695656
16 0.16382880876716888
17 0.1607156112893179
18 0.15981596663240857
19 0.1593627478515089
20 0.16000191045873485
21 0.15847747003284132
22 0.15740186592026545
23 0.1584278540877191
24 0.15643512802750698
25 0.15744070579912586
26 0.1566834033765046
27 0.15604189967611443
28 0.15538930024863348
29 0.15668173678837746
30 0.1544522404287879
31 0.15462889598365948
32 0.15651021802231022
33 0.1524547578479815
34 0.15208235370444334
35 0.15218995340359895
36 0.15196134878889375
37 0.15186875602414485
38 0.15370693974447228


56 0.14473035722058045
57 0.15202368953423537
58 0.1525263675024711
59 0.14822664664241048
60 0.15161597261374646
61 0.1459868436362668
62 0.1445371807711162
63 0.14542831091629893
64 0.14465011542611442
65 0.14481262636588144
66 0.14433777464826117
67 0.1447650020721225
68 0.14641365243836119
69 0.14594074577579033
70 0.14772549325659165
71 0.14316446066138303
72 0.14328800663057523
73 0.14343634612578532
74 0.14352605983894687
75 0.14384207650945474
76 0.1454336680405637
77 0.14387545370286986
78 0.14183563700998608
79 0.15985080385838338
80 0.1465696057746474
81 0.14369597617875446
82 0.14277007432926903
83 0.14544465937633289
84 0.14189532533391072
85 0.14935889143437853
86 0.14414297445666177
87 0.14300993522445085
88 0.1407506226852772
89 0.13939997191529854
90 0.13776805767045897
91 0.1394479979079938
92 0.14815787671671318
93 0.14202646239708536
94 0.1384186294618184
95 0.1409798416194944
96 0.13941370600488465
97 0.1363689545424206
98 0.13562289078097806
99 0.1360646045015823


## 4. Test on validation

In [ ]:
def test(model: torch.nn.Module, dataloader: torch.utils.data.DataLoader):

    model.eval()
    losses = []
    
    with torch.no_grad():
        for data, target in dataloader:
            # Forward pass
            out = model(data)
            
            losses.append(loss_fn(out,target).item())
        ### END CODE HERE ###
            
    return np.mean(losses)


In [ ]:
val_loss = test(model, dataload_val)
val_loss

0.19423764184117318

## 5. Export Results

'NonLinearNet'

In [ ]:
model_name = model.__repr__().split('(')[0]

y_test = model(torch.Tensor(X_test))
df_out = pd.DataFrame(y_test.detach().numpy())
df_out.columns = ['sat1_col']
df_out.to_csv(f"submission_Joking_{model_name}.csv")

### 6. Saving results for future purpose


In [ ]:
f= open("history.txt", "a+")
f.write(f"""
------------

{model}

Columns : {columns}
VAL LOSS : {val_loss}""")
f.close()

## 7. Exploring weights : identifying which cols are used

In [38]:
weightss = torch.Tensor(list(one_layer_net.parameters())[0]).detach().numpy().reshape((-1))
print(weightss)
print(cols_map[np.argsort(np.abs(weightss))])

[ 4.89094518e-02  1.60084367e-02 -2.99319699e-02 -8.15291852e-02
  6.07623123e-02 -6.49279952e-02  4.85553816e-02  5.62836509e-03
  3.26744094e-02  8.44709203e-03 -6.66475818e-02  2.06933171e-02
 -4.77486402e-02  5.48647456e-02  5.33154309e-02  1.52440378e-02
 -3.87026637e-04 -8.06853771e-02  5.62873706e-02  1.34960981e-02
 -4.31915820e-02 -4.82743699e-03 -6.04452658e-03 -2.33050920e-02
  7.61467144e-02 -5.38492650e-02 -4.01889682e-02  5.47575504e-02
 -6.48575090e-03 -1.81960166e-02 -2.60743331e-02  4.80739819e-03
  4.69803140e-02  6.28506318e-02 -1.28388451e-02 -3.03264260e-02
  8.63663014e-03  4.29149764e-03  5.32592274e-02 -6.76814541e-02
  1.20348493e-02 -4.98800166e-03  2.73799691e-02  3.26566100e-02
  4.12144847e-02 -4.27700616e-02 -3.59871611e-02  1.86313335e-02
  4.96571958e-02  4.93984185e-02 -2.62209494e-02 -6.89967796e-02
  8.12606364e-02  8.00067838e-03  3.11100036e-02  9.15166643e-03
  4.64990083e-03  4.19492237e-02 -1.76704526e-02  3.52022611e-02
  1.71438996e-02 -3.01328

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ad5b9508-4536-4617-b060-a55c942dfcf5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>